### Try to use LinearMap with ismutating=true, fail

In [ ]:
using LinearMaps
using IterativeSolvers

In [ ]:
function g(y::AbstractVector,x::AbstractVector) 
    y = [1 2;3 5]*x
    return y
end

S = LinearMap(g,2;ismutating=true)
b = [4.0, 3.3]
x = [2.0, 2.0]
gmres!(x,S,b)

### Try to use LinearMap with ismutating=false, works

In [ ]:
function f(x::AbstractVector) 
    return [1 2;3 5]*x
end

S = LinearMap(f,2;ismutating=false)
b = [4.0, 3.3]
gmres(S,b)

In [ ]:
[1 2;3 5]\[4.,3.3]

### Construct block matrix with functions, works

In [ ]:
m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]

[m1 m2;m3 m4]\b

In [ ]:
function al(x::AbstractVector)

m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]

    
    function f(x::AbstractVector) 
        return m1*x
    end
    function g(x::AbstractVector) 
        return m2*x
    end
    function m(x::AbstractVector) 
        return m3*x
    end
    function n(x::AbstractVector) 
        return m4*x
    end 
    
    # has to do the matrix mul manually to return 1d output
    out = zeros(5)
    out[1:3] = f(x[1:3]) + g(x[4:5])
    out[4:5] = m(x[1:3]) + n(x[4:5])
    
    return out
end
S = LinearMap(al,5;ismutating=false)    
    
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]
gmres(S,b)

### Test SaddleSystem

In [1]:
include("../src/SaddlePointSystems.jl")
using SaddlePointSystems

In [2]:
A = rand(80,80)
A⁻¹ = inv(A)
B₁ᵀ = rand(80,12)
B₂ = rand(12,80)
M = rand(12,12)
M⁻¹ = inv(M)
G₁ᵀ = rand(12,12)
G₂ = rand(12,12)
T₁ᵀ = rand(12,12)
T₂ = rand(12,12)

O32 = zeros(80,12)
O23 = zeros(12,80)
O22 = zeros(12,12)

S = [A O32 B₁ᵀ O32; O23 M T₁ᵀ G₁ᵀ; B₂ -T₂ O22 O22; O23 G₂ O22 O22]

rċ = rand(80)
ru̇ = rand(12)
rf = rand(12)
rλ = rand(12)
b = [rċ;ru̇;rf;rλ]

S\b

116-element Array{Float64,1}:
   1.50932 
   1.02995 
  -3.08119 
  -0.954666
  -4.72321 
  -3.66929 
  -1.47647 
   1.05003 
   0.568912
   0.383864
   0.520295
   2.43736 
   1.10231 
   ⋮       
 -63.1032  
 102.057   
 -66.5229  
 -18.8791  
 -40.6985  
  11.8923  
  65.1501  
 -74.92    
  -2.13936 
 -33.8822  
 126.728   
  45.4167  

In [3]:
norm(S*(S\b)-b)

1.6612005790442985e-13

In [4]:
rank(S) == 116

true

In [5]:
ċ = zeros(80)
u̇ = zeros(12)
f = zeros(12)
λ = zeros(12)

St = SaddleSystem((ċ, u̇, f, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M⁻¹, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x))

bt = (rċ, ru̇, rf, rλ);

In [6]:
aa,bb,cc,dd = St\bt

([0.726642, 0.618506, 0.724247, 0.38735, 0.470641, 0.731597, -0.505022, -0.495756, 0.128154, -0.0313404  …  -0.295983, 0.829808, -0.723865, -0.189682, -0.513348, -0.79321, 0.655941, -0.0661136, -0.124525, 0.15814], [-0.561615, -0.758761, 0.534764, 0.14125, 0.6064, -0.292093, 0.103801, 0.134363, 0.604613, 0.506219, -0.641787, 0.403854], [0.536129, -0.401146, -0.235803, 0.78067, 0.255255, 0.401419, -0.62165, -0.351508, 0.20378, -0.293128, -0.0152623, -1.10706], [-1.39898, -0.731414, 0.547435, 0.964782, 0.335464, 0.889258, -0.689853, 0.304351, 0.26564, 0.0126426, -0.336897, -0.433978])

In [7]:
x = [aa;bb;cc;dd]
norm(S*x-b)

1.3001905157452815